In [10]:
# Import libraries and packages for the project 

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import csv

print('- Finish importing packages')

- Finish importing packages


In [11]:
# Task 1: Login to Linkedin

# Task 1.1: Open Chrome and Access Linkedin login site
driver = webdriver.Chrome("/home/congtu/Documents/Hoc/XuLyDuLieuLon/BTL/crawl_data_estate/chromedriver")
driver.set_window_size(1920,1080)
sleep(2)
url = 'https://batdongsan.com.vn/ban-nha-rieng'
driver.get(url)
print('- Finish initializing a driver')
sleep(2)

/home/congtu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  after removing the cwd from sys.path.


- Finish initializing a driver


In [12]:
# Task 3: Scrape the URLs of the profiles
# Task 3.1: Write a function to extract the URLs of one page
def GetURL():
    page_source = BeautifulSoup(driver.page_source, "html.parser")
    profiles = page_source.find_all('a', class_ = 'js__product-link-for-product-id') #('a', class_ = 'search-result__result-link ember-view')
    all_profile_URL = []
    for profile in profiles:
        # profile_ID = profile.get('href')
        profile_URL = "https://batdongsan.com.vn" + profile.get('href')
        # profile_URL = profile.get('href')
        if profile_URL not in all_profile_URL:
            all_profile_URL.append(profile_URL)
    return all_profile_URL


# Task 3.2: Navigate through many page, and extract the profile URLs of each page
input_page = int(input('How many pages you want to scrape: '))
URLs_all_page = []
for page in range(input_page):
    URLs_one_page = GetURL()
    sleep(2)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') #scroll to the end of the page
    sleep(3)
    next_button = driver.find_element_by_class_name("re__icon-chevron-right--sm")
    driver.execute_script("arguments[0].click();", next_button)
    URLs_all_page = URLs_all_page + URLs_one_page
    sleep(2)
print('- Finish Task 3: Scrape the URLs')

How many pages you want to scrape: 1


/home/congtu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


- Finish Task 3: Scrape the URLs


In [13]:
def area_cal(a):
    a = a.lower()
    return float(a.split('m')[0].strip().replace(',', '.'))
def price_cal(p, a):
    p = p.lower()
    if ('/m' in p):
        if a == 0:
            return 0
        else:
            return float(p.split('triệu')[0].strip().replace(',', '.')) * a/1000
    if ('triệu' in p):
        return float(p.split('triệu')[0].strip().replace(',', '.'))/1000
    if ('tỷ' in p):
        return float(p.split('tỷ')[0].strip().replace(',', '.'))
    return none

In [22]:
# Task 4: Scrape the data of 1 Linkedin profile, and write the data to a .CSV file
with open('sell_house_villa.csv', 'w',  newline = '') as file_output:
    headers = ['name', 'address', 'city', 'district', 'price (tỷ)', 'area (m^2)', 'floor',
               'bedroom', 'post_date', 'expiration_date' ]
    writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n',fieldnames=headers)
    writer.writeheader()    
    for linkedin_URL in URLs_all_page:
        driver.get(linkedin_URL)
        print('- Accessing profile: ', linkedin_URL)

        page_source = BeautifulSoup(driver.page_source, "html.parser")

        info_div = page_source.find('div',{'class':'re__pr-info pr-info'})
        dc = page_source.find("div", {"class" : "re__section-body re__border--std js__section-body"}).find_all("div", {"class" : "re__list-standard-1line--md"})
        gia = page_source.find('div',{'class':'re__pr-short-info js__pr-short-info'}).find_all("div", {"class" : "re__pr-short-info-item js__pr-short-info-item"})          
        ngay = page_source.find('div',{'class':'re__pr-short-info re__pr-config js__pr-config'}).find_all("div", {"class" : "re__pr-short-info-item js__pr-config-item"})          

        try:
            name = info_div.find('h1', class_='re__pr-title pr-title').get_text().strip()  
            print('--- name is: ', name)       
            address = dc[1].find("span", {"class" : "value"}).get_text().strip()
            print('--- address is: ', address)
            li = address.split(',')
            city = li[-1].strip().strip('.')
            district = li[-2].strip()
            print('--- city is: ', city)
            print('--- district is: ', district)

            area = gia[1].find("span", {"class" : "value"}).get_text().strip()
            area = area_cal(area)
            print('--- area is: ', area)

            price = gia[0].find("span", {"class" : "value"}).get_text().strip()
            price = price_cal(price, area)
            print('--- price is : ', price)

            floor = dc[2].find("span", {"class" : "value"}).get_text().strip()
            floor = floor.split(' ')
            floor = floor[0]
            print('--- floor is: ', floor)
            
            bedroom = gia[2].find("span", {"class" : "value"}).get_text().strip()
            bedroom = bedroom.split(' ')
            bedroom = bedroom[0]
            print('--- bedroom is: ', bedroom)

            post_date = ngay[0].find("span", {"class" : "value"}).get_text().strip()
            expiration_date = ngay[1].find("span", {"class" : "value"}).get_text().strip()
            print('--- post date is: ', post_date)
            print('--- expiration date is: ', expiration_date)
            
            writer.writerow({headers[0]:name, headers[1]:address, headers[2]:city, headers[3]:district, 
                             headers[4]:price, headers[5]:area, headers[6]:bedroom,
                             headers[7]:bedroom, headers[8]:post_date, headers[9]:expiration_date})
            print('\n')
        except:
            pass
print('Mission Completed!')

- Accessing profile:  https://batdongsan.com.vn/ban-nha-rieng-phuong-16-3/ban-dep-co-ho-boi-trong-noi-that-xinh-dtsd-toi-281-5m2-nut-giao-tan-binh-quan-12-pr31566182
--- name is:  Bán nhà đẹp có hồ bơi trong nhà, nội thất Nhà Xinh. DTSD tới 281.5m2, nút giao Tân Bình, Quận 12
--- address is:  Phường 16, Gò Vấp, Hồ Chí Minh
--- city is:  Hồ Chí Minh
--- district is:  Gò Vấp
--- area is:  146.9
--- price is :  15.5
--- floor is:  3
--- bedroom is:  3
--- post date is:  24/11/2021
--- expiration date is:  24/12/2021


- Accessing profile:  https://batdongsan.com.vn/ban-nha-rieng-duong-nguyen-an-ninh-phuong-tuong-mai/cong-ty-c-b-toa-moi-mat-ngo-104-pho-60m2x7t-thg-may-gia-14-6-ty-co-h-pr31560342
--- name is:  CÔNG TY CẦN BÁN TÒA NHÀ MỚI MẶT NGÕ 104 PHỐ NGUYỄN AN NINH 60M2x7T THANG MÁY GIÁ 14,6 TỶ (CÓ ẢNH)
--- address is:  104 Nguyễn An Ninh, phường Tương Mai, quận Hoàng Mai, Thành Phố Hà Nội
--- city is:  Thành Phố Hà Nội
--- district is:  quận Hoàng Mai
--- area is:  60.0
--- price is :  

--- name is:  Chính chủ (không qua môi giới) bán nhà 5 tầng ngõ 252 Tây Sơn
--- address is:  Ngõ 252, Đường Tây Sơn, Phường Trung Liệt, Đống Đa, Hà Nội
--- city is:  Hà Nội
--- district is:  Đống Đa
--- area is:  42.0
--- price is :  4.6
--- floor is:  5
--- bedroom is:  4
--- post date is:  18/11/2021
--- expiration date is:  28/11/2021


- Accessing profile:  https://batdongsan.com.vn/ban-nha-rieng-phuong-bien-giang/ban-trung-tam-quan-ha-dong-gia-tu-1-65-ty-o-to-vao-pr31202412
--- name is:  Bán nhà trung tâm Phường Biên Giang, Quận Hà Đông giá từ 1.45 tỷ ô tô vào nhà, LH: 0934657***
--- address is:  Phường Biên Giang, Hà Đông, Hà Nội
--- city is:  Hà Nội
--- district is:  Hà Đông
--- area is:  30.7
--- price is :  1.45
--- floor is:  3.6
- Accessing profile:  https://batdongsan.com.vn/ban-nha-rieng-pho-lach-tray-phuong-dong-hai/canh-bao-ban-o-to-do-cua-54m2-x-4-tang-gia-2-5-ty-khong-mua-la-lo-pr31262897
--- name is:  Bán nhà Lạch Tray, ô tô đỗ cửa, 54m2 x 4 tầng. Giá 2,5 tỷ, không m